# Bayesian Blocks Analysis on a Light Curve

Bayesian Blocks, provide an adaptive, data-driven method for optimal segmentation of time series or event data.
Instead of relying on fixed bin sizes, Bayesian Blocks dynamically determine change points in the data, dividing it into piecewise constant segments (blocks) that best represent the structure of the signal.
We follow the formalization of [Scargle et al. (2013)](https://iopscience.iop.org/article/10.1088/0004-637X/764/2/167).

Key features of the Bayesian Blocks algorithm include:
- Model selection via Bayesian inference: The algorithm selects the most probable segmentation under a Bayesian framework, balancing goodness-of-fit with model complexity.
- Applicability to different data modes: It works with event data (e.g., photon arrival times), regularly sampled time series, and point measurements with errors.
- Non-parametric nature: The method does not assume a particular functional form for the signal, making it robust and flexible.



`Agilepy` implements a `AGBayesianBlocks` class that works with:
- Binned data, i.e. Light Curves. Input data can be provided in 3 different formats: the agile [Aperture Photometry Light Curve](#Bayesian-Blocks-on-Aperture-Photometry-Light-Curves) format, the agile [Maximum Likelihood Light Curve](#Bayesian-Blocks-on-Maximum-Likelihood-Light-Curves) format, a *custom* light curve format.
- [Unbinned](#Bayesian-Blocks-on-Photon-Lists) data, i.e. a list of arrival times of detected photons.

The Bayesian Blocks algorithm has the following steps:
1. Prepare a `YAML` configuration file for `AGBayesianBlocks`. You can do it manually or by using `AGBayesianBlocks.getConfiguration()`.
2. Define a `AGBayesianBlocks` object, and load the data with `AGBayesianBlocks.selectEvents()`.
3. Compute the Bayesian blocks with `AGBayesianBlocks.bayesianBlocks()`.

Note: the custom light curve format is a text file with three columns: bin start time, bin stop times, flux.
Columns do not need an header and must be separated by white spaces.

In [ ]:
# Import the relevant classes
import agilepy

from agilepy.api.AGBayesianBlocks import AGBayesianBlocks


## Bayesian Blocks on Aperture Photometry Light Curves

Aperture Photometry Light Curves can be obtained with `AGAnalysis.aperturePhotometry()`.

They are text files with no header and 4 columns separated by white spaces: bin start time, bin stop time, on-source exposure, on-source counts.

In [ ]:
# Prepare the Configuration File.
# You can create it manually or use AGBayesianBlocks.getConfiguration

outputDir_ap    = "/home/flareadvocate/workspace/shared_dir/"
confFilePath_ap = "/home/flareadvocate/workspace/shared_dir/tutorial_bb_ap.yaml"

AGBayesianBlocks.getConfiguration(confFilePath=confFilePath_ap,
                                  outputDir=outputDir_ap,
                                  userName="my-name",
                                  sourceName="bb-source",
                                  verboselvl=0,
                                  filePath="$AGILE/agilepy-test-data/bayesian_blocks/3C454.3_2010flare_86400s.ap",
                                  fileMode="AGILE_AP",
                                  tstart="null",
                                  tstop="null",
                                  rateCorrection=0,
                                  fitness="events",
                                  gamma=0.35,
                                  p0="null",
                                  useerror="true"
                                  )

The `YAML` file has the following sections and field:
- `output`: common to all `agilepy` classes, sets the output fields
- `selection`: include arguments used by `AGBayesianBlocks.selectEvents()` used to prepare the data:
    - `filePath`: path to the input file
    - `fileMode`: defines the input file format: `AGILE_AP`, `AGILE_MLE`, `CUSTOM_LC` (binned) or `AGILE_PH` (unbinned).
    - `tstart`, `tstop`: start and stop times for data selection within the input file. For `AGILE_AP` and `AGILE_MLE` must be in `MJD` format, for `CUSTOM_LC`, `AGILE_PH` in the same time format of the times provided.
    - `ratecorrection`: the Bayesian Blocks algorithm works with integers, so fluxes need a correction factor for a correct statistical treatment. If 0 (default), the correction factor is the mean exposure. If a float is provided it will be applied as a correction factor for the flux. If -1, the algorithm will work with counts instead of fluxes.
- `bayesianblocks`: arguments used by `AGBayesianBlocks.bayesianBlocks()` to compute the blocks:
    - `fitness`: set the type of algorithm. Possible values are `events` (binned light curve or unbinned event data), `measures` (sequence of flux measurements with Gaussian errors), `regular_events`(0/1 data measured regularly).
    - `useerror`: boolean to include errors in the computation of the blocks.
    - `gamma`, `p0`: arguments on the prior. `p0`, suggested for unbinned data, gives the false alarm probability to compute the prior. `gamma`, suggested for binned data, gives the slope of the prior on the number of bins.



In [ ]:
# Create the AGBayesianBlocks object
ag_bb_ap = AGBayesianBlocks(confFilePath_ap)

In [ ]:
# You can Use the setOptions() method to change the configuration, e.g.:
ag_bb_ap.setOptions(gamma=0.35)

# Check configuration.
ag_bb_ap.printOptions()


In [ ]:
# Select Events
# If no argument is provided, they are read from the configuration file.
ag_bb_ap.selectEvents()

In [ ]:
# You can Inspect the selected data with getDataIn()
selected_events_ap = ag_bb_ap.getDataIn()

In [ ]:
# Print
display(selected_events_ap)

# Plot the Data
ag_bb_ap.plotBayesianBlocks()

In [ ]:
# Compute Bayesian Blocks
ag_bb_ap.bayesianBlocks()

In [ ]:
# You can Inspect the results with getDataout()
blocks_computed_ap = ag_bb_ap.getDataOut()

In [ ]:
# Print
display(blocks_computed_ap.keys())
display(len(blocks_computed_ap['data_cells']))
display(len(blocks_computed_ap['blockrate']))
display(blocks_computed_ap['N_change_points'])

display(blocks_computed_ap)


In [ ]:
# Plot
ag_bb_ap.plotBayesianBlocks(saveImage=False, plotRate=True, plotDataCells=True)

## Bayesian Blocks on Maximum Likelihood Light Curves
Maximum Likelihood Light Curves can be obtained with `AGAnalysis.lightCurve()`.

They are text files with header and multiple columns separated by white spaces.

In [ ]:
# Prepare the Configuration File.
# You can create it manually or use AGBayesianBlocks.getConfiguration

outputDir_mle    = "/home/flareadvocate/workspace/shared_dir/"
confFilePath_mle = "/home/flareadvocate/workspace/shared_dir/tutorial_bb_mle.yaml"

AGBayesianBlocks.getConfiguration(confFilePath=confFilePath_mle,
                                  outputDir=outputDir_mle,
                                  userName="my-name",
                                  sourceName="bb-source",
                                  verboselvl=0,
                                  filePath="$AGILE/agilepy-test-data/bayesian_blocks/3C454.3_2010flare_86400s_lc_mle.txt",
                                  fileMode="AGILE_MLE",
                                  tstart="null",
                                  tstop="null",
                                  rateCorrection=0,
                                  fitness="events",
                                  gamma=0.35,
                                  p0="null",
                                  useerror="true"
                                  )

In [ ]:
# Create the AGBayesianBlocks object
ag_bb_mle = AGBayesianBlocks(confFilePath_mle)

# You can Use the setOptions() method to change the configuration, e.g.:
# ag_bb_mle.setOptions(gamma=0.35)

# Check configuration.
ag_bb_mle.printOptions()


In [ ]:
# Select Events
ag_bb_mle.selectEvents()
# Inspect
selected_events_mle = ag_bb_mle.getDataIn()

In [ ]:
# Compute Bayesian Blocks
ag_bb_mle.bayesianBlocks()
# Inspect
blocks_computed_mle = ag_bb_mle.getDataOut()

In [ ]:
# Print
display(blocks_computed_mle['N_change_points'])

# Plot
ag_bb_mle.plotBayesianBlocks(saveImage=False, plotRate=True, plotDataCells=True)

## Bayesian Blocks on Photon Lists
Photon list files are text files separated by whitespaces where the first column is the photon arrival times.
Please perform any other selection (e.g. position, energy, data quality) beforehand.

In [ ]:
# Prepare the Configuration File.
# You can create it manually or use AGBayesianBlocks.getConfiguration

outputDir_ph    = "/home/flareadvocate/workspace/shared_dir/"
confFilePath_ph = "/home/flareadvocate/workspace/shared_dir/tutorial_bb_ph.yaml"
filePath_ph = agilepy.__path__[0]+"/testing/unittesting/api/data/test_data.ph"

AGBayesianBlocks.getConfiguration(confFilePath=confFilePath_ph,
                                  outputDir=outputDir_ph,
                                  userName="my-name",
                                  sourceName="bb-source",
                                  verboselvl=0,
                                  filePath=filePath_ph,
                                  fileMode="AGILE_PH",
                                  tstart="null",
                                  tstop="null",
                                  rateCorrection=0,
                                  fitness="events",
                                  gamma="null",
                                  p0=0.5,
                                  useerror="true"
                                  )


In [ ]:
# Create the AGBayesianBlocks object
ag_bb_ph = AGBayesianBlocks(confFilePath_ph)

# Check configuration.
ag_bb_ph.printOptions()

In [ ]:
# Select Events
ag_bb_ph.selectEvents()

# Inspect
selected_events_ph = ag_bb_ph.getDataIn()

In [ ]:
# Compute Bayesian Blocks
ag_bb_ph.bayesianBlocks()

# Inspect
blocks_computed_ph = ag_bb_ph.getDataOut()

In [ ]:
# Print
display(blocks_computed_ph['N_change_points'])

# Plot
ag_bb_ph.plotBayesianBlocks(saveImage=False, plotRate=True)